# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

In [ ]:
sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston algorithm

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [ ]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [ ]:
list_params = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [ ]:
list_params

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 50
validation_training_ratio = 0.95

# Pipeline for Croston

In [ ]:
def Croston_output_all_params(sales_pattern, list_params, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in range(len(train_second)):
                yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                            'Actual Data': train_second,
                                            'Forecast': predictions,
                                            'Alpha': [i for count in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
start = timer()

pattern_df_result_all_params = Croston_output_all_params(sales_smooth,
                                                      list_params,
                                                      start_train_date,
                                                      end_train_date, 
                                                      n_pred_days,
                                                      num_train_needed,
                                                      validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 2.671021005816667 minutes


In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,FOODS_1_004_CA_1_validation,7.0,6.442068,0.1
1,FOODS_1_004_CA_1_validation,11.0,6.475027,0.1
2,FOODS_1_004_CA_1_validation,7.0,6.753741,0.1
3,FOODS_1_004_CA_1_validation,7.0,6.769515,0.1
4,FOODS_1_004_CA_1_validation,11.0,6.784829,0.1
...,...,...,...,...
213781,FOODS_3_818_CA_4_validation,5.0,1.743312,0.9
213782,FOODS_3_818_CA_4_validation,2.0,4.648159,0.9
213783,FOODS_3_818_CA_4_validation,0.0,2.266961,0.9
213784,FOODS_3_818_CA_4_validation,3.0,2.266961,0.9


In [ ]:
pattern_df_result_all_params.to_csv('Croston_Smooth_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summary all metrics

In [ ]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.24180492379999993 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_1_validation,0.1,0.684121,52.7344,61.1843,94.2818,2.907516,3.680946
1,FOODS_1_004_CA_1_validation,0.2,0.694550,53.5382,62.0084,94.5207,2.951836,3.790882
2,FOODS_1_004_CA_1_validation,0.3,0.711936,54.8784,63.0335,100.2210,3.025728,3.905050
3,FOODS_1_004_CA_1_validation,0.4,0.745083,57.4335,65.5544,106.9484,3.166604,4.013299
4,FOODS_1_004_CA_1_validation,0.5,0.777390,59.9238,67.9023,112.6659,3.303909,4.120481
...,...,...,...,...,...,...,...,...
5773,FOODS_3_823_CA_3_validation,0.5,inf,inf,200.0000,NaN,0.755952,0.755952
5774,FOODS_3_823_CA_3_validation,0.6,inf,inf,200.0000,NaN,0.760486,0.760486
5775,FOODS_3_823_CA_3_validation,0.7,inf,inf,200.0000,NaN,0.774115,0.774115
5776,FOODS_3_823_CA_3_validation,0.8,inf,inf,200.0000,NaN,0.790884,0.790884


# Check what products has unexpected metrics result

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
531,FOODS_2_024_CA_2_validation,0.1,NaN,NaN,NaN,NaN,0.000000,0.000000
532,FOODS_2_024_CA_2_validation,0.2,NaN,NaN,NaN,NaN,0.000000,0.000000
533,FOODS_2_024_CA_2_validation,0.3,NaN,NaN,NaN,NaN,0.000000,0.000000
534,FOODS_2_024_CA_2_validation,0.4,NaN,NaN,NaN,NaN,0.000000,0.000000
535,FOODS_2_024_CA_2_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
5773,FOODS_3_823_CA_3_validation,0.5,inf,inf,200.0,NaN,0.755952,0.755952
5774,FOODS_3_823_CA_3_validation,0.6,inf,inf,200.0,NaN,0.760486,0.760486
5775,FOODS_3_823_CA_3_validation,0.7,inf,inf,200.0,NaN,0.774115,0.774115
5776,FOODS_3_823_CA_3_validation,0.8,inf,inf,200.0,NaN,0.790884,0.790884


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
531,FOODS_2_024_CA_2_validation,0.1,NaN,NaN,NaN,NaN,0.000000,0.000000
532,FOODS_2_024_CA_2_validation,0.2,NaN,NaN,NaN,NaN,0.000000,0.000000
533,FOODS_2_024_CA_2_validation,0.3,NaN,NaN,NaN,NaN,0.000000,0.000000
534,FOODS_2_024_CA_2_validation,0.4,NaN,NaN,NaN,NaN,0.000000,0.000000
535,FOODS_2_024_CA_2_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
5773,FOODS_3_823_CA_3_validation,0.5,inf,inf,200.0,NaN,0.755952,0.755952
5774,FOODS_3_823_CA_3_validation,0.6,inf,inf,200.0,NaN,0.760486,0.760486
5775,FOODS_3_823_CA_3_validation,0.7,inf,inf,200.0,NaN,0.774115,0.774115
5776,FOODS_3_823_CA_3_validation,0.8,inf,inf,200.0,NaN,0.790884,0.790884


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
531,FOODS_2_024_CA_2_validation,0.1,NaN,NaN,NaN,NaN,0.000000,0.000000
532,FOODS_2_024_CA_2_validation,0.2,NaN,NaN,NaN,NaN,0.000000,0.000000
533,FOODS_2_024_CA_2_validation,0.3,NaN,NaN,NaN,NaN,0.000000,0.000000
534,FOODS_2_024_CA_2_validation,0.4,NaN,NaN,NaN,NaN,0.000000,0.000000
535,FOODS_2_024_CA_2_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
5638,FOODS_3_795_CA_4_validation,0.5,0.711895,85.3615,NaN,73.0545,0.553696,0.980300
5639,FOODS_3_795_CA_4_validation,0.6,0.713343,85.5351,NaN,70.7531,0.554822,0.968986
5640,FOODS_3_795_CA_4_validation,0.7,0.735837,88.2323,NaN,72.5833,0.572318,0.968875
5641,FOODS_3_795_CA_4_validation,0.8,0.743145,89.1086,NaN,72.3402,0.578002,0.971926


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
531,FOODS_2_024_CA_2_validation,0.1,NaN,NaN,NaN,NaN,0.000000,0.000000
532,FOODS_2_024_CA_2_validation,0.2,NaN,NaN,NaN,NaN,0.000000,0.000000
533,FOODS_2_024_CA_2_validation,0.3,NaN,NaN,NaN,NaN,0.000000,0.000000
534,FOODS_2_024_CA_2_validation,0.4,NaN,NaN,NaN,NaN,0.000000,0.000000
535,FOODS_2_024_CA_2_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...
5773,FOODS_3_823_CA_3_validation,0.5,inf,inf,200.0,NaN,0.755952,0.755952
5774,FOODS_3_823_CA_3_validation,0.6,inf,inf,200.0,NaN,0.760486,0.760486
5775,FOODS_3_823_CA_3_validation,0.7,inf,inf,200.0,NaN,0.774115,0.774115
5776,FOODS_3_823_CA_3_validation,0.8,inf,inf,200.0,NaN,0.790884,0.790884


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 10.903426791277258 %
Percentage of unexpected values of WMAPE is: 10.903426791277258 %
Percentage of unexpected values of SMAPE is: 13.084112149532709 %
Percentage of unexpected values of MAPE is: 10.903426791277258 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.921083,79.143290,76.914129,63.175890,3.865390,4.929453
0.2,0.904928,75.321528,77.056822,64.731322,3.825305,4.927349
0.3,0.914597,75.171758,77.792108,66.593220,3.865378,4.991239
0.4,0.930920,75.968539,78.660983,68.420148,3.922538,5.073129
0.5,0.950215,77.178414,79.613494,70.295035,3.986588,5.161969
0.6,0.971242,78.636495,80.644123,72.258601,4.057679,5.257884
0.7,0.993455,80.272203,81.768579,74.350503,4.133043,5.364914
0.8,1.017155,82.075639,83.045917,76.631190,4.215718,5.488645
0.9,1.043264,84.138317,84.542646,79.219249,4.314221,5.636861


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.2
The optimum Alpha based on Mean WMAPE is: 0.3
The optimum Alpha based on Mean SMAPE is: 0.1
The optimum Alpha based on Mean MAPE is: 0.1
The optimum Alpha based on Mean MAE is: 0.2
The optimum Alpha based on Mean RMSE is: 0.2


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.800696,58.86795,66.95580,57.44075,2.381073,3.041756
0.2,0.808739,59.50250,67.92255,59.63865,2.429241,3.078459
0.3,0.825258,60.60040,68.43215,62.37015,2.497730,3.143950
0.4,0.847938,62.00000,69.40440,64.52165,2.570214,3.220504
0.5,0.867791,63.63830,70.45380,66.56050,2.629446,3.303498
0.6,0.884862,64.49460,72.13590,68.28370,2.689060,3.387796
0.7,0.907538,65.78150,73.66990,70.63615,2.725753,3.476227
0.8,0.931832,66.68785,75.30690,72.41815,2.813800,3.577599
0.9,0.962097,68.72195,77.28095,74.24870,2.918368,3.698474


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.1
The optimum Alpha based on Median of WMAPE is: 0.1
The optimum Alpha based on Median of SMAPE is: 0.1
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.1
The optimum Alpha based on Median of RMSE is: 0.1


# Run Croston model after decide best paramaters

In [ ]:
best_alpha = 0.2

In [ ]:
def Croston_test_data(best_alpha, sales_pattern, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [ ]:
start = timer()

pattern_df_result_best_params = Croston_test_data(best_alpha, sales_smooth, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.11505257648333327 minutes


In [ ]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [ ]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_1_validation,0.726688,63.7979,71.7918,85.3886,4.192433,4.909072
1,FOODS_1_004_CA_3_validation,inf,inf,200.0000,NaN,1.986644,1.986644
2,FOODS_1_004_CA_4_validation,0.817757,49.2134,52.6400,63.1852,1.195183,1.615928
3,FOODS_1_012_CA_1_validation,0.706850,49.2556,56.1495,73.7419,1.794312,2.233910
4,FOODS_1_012_CA_2_validation,0.684937,40.2782,44.2880,35.2440,4.373058,5.122002
...,...,...,...,...,...,...,...
637,FOODS_3_818_CA_4_validation,0.562415,41.7709,46.3824,40.1641,0.865255,1.172941
638,FOODS_3_819_CA_3_validation,0.713802,57.2765,56.1143,115.3239,2.086499,2.396190
639,FOODS_3_820_CA_1_validation,0.578268,54.7265,65.3397,48.9039,1.289983,1.538355
640,FOODS_3_820_CA_3_validation,0.718231,74.1251,88.4180,44.9566,1.270717,1.501813


In [ ]:
df_result_final.isnull().sum()

Product     0
MASE       65
WMAPE      65
SMAPE      67
MAPE       76
MAE         0
RMSE        0
dtype: int64

In [ ]:
df_result_final.to_csv('Croston_Smooth_Test_Data.csv')